In [6]:
from math import gamma

import numpy as np
import math
import matplotlib.pyplot as plt
import sympy as sp

In [7]:
print(1.0073-0.906569406021549)
print(1.10802927402634-1.0073)



0.10073059397845108
0.10072927402633991


In [8]:
solution3 = sp.solve(-0.75 + Cd * A * sp.sqrt(2*rho_ox*P),P)
print(solution3)
print(5.4e5 - solution3[0])


[425606.719718490]
114393.280281510


In [9]:
m_dot, P = sp.symbols("m_dot, P")

ve = 3145.7598
Ae = 0.1527

Me = 4.380
gamma_c = 1.1173
gamma_e = 1.213
#gamma_avg = (gamma_c + gamma_e)/2
gamma_avg = 1.1414
Te = 1286.42
F = 3304.6615
c_star = 1605
At = 1796.285e-6



m_dot_ox_norm = 0.8448
m_dot_prop_norm = 1.0073

Cd = 0.8
rho_ox = 1573.2
A = m_dot_ox_norm/Cd/math.sqrt(2*rho_ox*5.4e5)
rho_f = 1522
Dpi = 95.6473e-3
L = 515.9178e-3

Thrust_Percent = 1.1

count = 0
while True:
    if count == 0:
        r = 6.35e-05*(m_dot_ox_norm/(math.pi/4*Dpi**2))**0.5
        m_dot_f = rho_f*math.pi*Dpi*L*r
        m_dot_f_norm = m_dot_f
        OF = m_dot_ox_norm/m_dot_f
        Pe = (1+(gamma_avg-1)/2*Me**2)**(-gamma_avg/(gamma_avg-1))*m_dot*(-2.97619*OF**2 + 10.05952*OF + 1633.18571)/At
        m_dot_prop_update = sp.solve(-Thrust_Percent*F + m_dot * ve + Pe * Ae, 'm_dot')
        m_dot_ox_new = abs(m_dot_ox_norm - abs(m_dot_prop_norm - m_dot_prop_update[0]))
        m_dot_ox_old = m_dot_ox_norm
        m_dot_f_old = m_dot_f
        Pe_norm = Pe.subs(m_dot, m_dot_prop_update[0])
        count += 1
        continue
        
    r = 6.35e-05*(m_dot_ox_new/(math.pi/4*Dpi**2))**0.5
    m_dot_f = rho_f*math.pi*Dpi*L*r
    #m_dot_prop_new = m_dot_ox_new + m_dot_f
    OF = m_dot_ox_new/m_dot_f
    Pe = (1+(gamma_avg-1)/2*Me**2)**(-gamma_avg/(gamma_avg-1))*m_dot*(-2.97619*OF**2 + 10.05952*OF + 1633.18571)/At
    m_dot_prop_update = sp.solve(-Thrust_Percent*F + m_dot * ve + Pe * Ae, 'm_dot')
    m_dot_ox_new = abs(m_dot_prop_update[0] - m_dot_f)
    
    if count != 0 and round(abs(m_dot_f - m_dot_f_old),5) == 0 and round(abs(m_dot_ox_new - m_dot_ox_old),5) == 0 and round(abs(m_dot_ox_new + m_dot_f - m_dot_prop_update[0]),5) == 0:
        break
    
    #if count != 0 and round(m_dot_f - m_dot_f_old,10) != 0 and round(m_dot_ox_new - m_dot_ox_old,10) != 0 and round(m_dot_ox_new + m_dot_f - m_dot_prop_update[0],10) != 0:
    else:
        m_dot_ox_old = m_dot_ox_new
        m_dot_f_old = m_dot_f
        count += 1
    
#print(round(m_dot_ox_new + m_dot_f - m_dot_prop_update[0],10))
print(count)   
print(f"Oxidizer Mass Flow Rate = {m_dot_ox_new:.6f} kg/s")
print(f"Fuel Mass Flow Rate Norm = {m_dot_f_norm:.6f} kg/s")
print(f"Fuel Mass Flow Rate Now = {m_dot_f:.6f} kg/s")
print(f"Propellant Mass Flow Rate = {m_dot_prop_update[0]:.6f} kg/s")
print(f"Propellant Mass Flow Uncertainty = {m_dot_prop_norm - m_dot_prop_update[0]:.12f} kg/s")
print(f"Current O/F = {OF}")
print(f"{Thrust_Percent*100:.0f}% Thrust = {Thrust_Percent*F:.4f}")
print(f"Current C* = {-2.97619*OF**2 + 10.05952*OF + 1633.18571:.4f}")
Pe = (1+(gamma_avg-1)/2*Me**2)**(-gamma_avg/(gamma_avg-1))*m_dot_prop_update[0]*(-2.97619*OF**2 + 10.05952*OF + 1633.18571)/At
print(f"Calculated Thrust = {m_dot_prop_update[0] * ve + Pe * Ae:.4f}")
m_dot_ox_change = m_dot_ox_norm - m_dot_ox_new
print(f"Oxidizer Mass Flow Rate Uncertainty = {m_dot_ox_change:.12f} kg/s")
pressure_new = sp.solve(-m_dot_ox_new + Cd * A * sp.sqrt(2*rho_ox*P),P)
print(f"Pressure drop in injector = {pressure_new[0]:.2f} Pa")
print(f"Pressure Rise Uncertainty = {(5.4e5 - pressure_new[0])/1e5:.2f} Bar")
print(A)
print(f"Nominal Pe = {Pe_norm}")
print(f"Pe Now = {Pe.subs(m_dot, m_dot_prop_update[0])}")

KeyboardInterrupt: 